This workbook is to create tickets for Connect Remastering for Audit, SAQ, and map them to the placeholder release milestone(s)

Should just be able to hit "Run all"

In [32]:
import pandas as pd
import json

pd.options.display.max_rows = 999
import numpy as np
import re
import pickle
from atlassian import Jira

import networkx as nx
import plotly.graph_objects as go
from plotly.validators.scatter.marker import SymbolValidator
from pyvis.network import Network 
import pyvis.options as pyvis_options
import datetime

with open(r'C:\Users\steve.waterman\Python\creds\Jira', 'rb') as handle:
    mycon = pickle.load(handle)
exec(f'jira = Jira({mycon})')
from pprint import pprint

import sys
sys.path.append(r'C:\Users\steve.waterman\Python\read_bitbucket')
import lineage_vis as lv
import Useful_Jira as uj

## Get the JIRA board for Data Platform

In [33]:
# Get project and board
myproj,myboard=uj.myprojectAndBoard(jira,'Data Platform',218)

# Get tickets
mytickets=uj.get_all_project_issues_uncapped(jira,myproj['id'])

# Summarise tickets (removing unused fields)
mytickets_succinct=uj.get_succint_ticket_list(mytickets)

## Map Epics to their Macro strategy strands

In [34]:
board_epics,epic_name_list, epic_key_list,board_epic_key_lookup=uj.get_epics_only(mytickets_succinct)

In [35]:
epic_name_list

['Data Readiness: General',
 'Platform: General Tasks',
 'Platform: Permissions',
 'Data Readiness: PDF Reader Domain',
 'Platform: Data-Lineage',
 'Data Readiness: Discovery & Profiling',
 'Platform: Data-Alerting-&-Diagnostics',
 'Platform: Data-Unit-Tests & Deployment ',
 'Platform: GDPR+',
 'Data Readiness:  ethnet Domain',
 'Data Readiness:  user-management Domain',
 'Data Readiness:  RefData Domain',
 'Data Readiness:  slcp-assessments Domain',
 'Data Readiness:  Visibility Domain',
 'Data Readiness:  saq Domain',
 'Data Readiness:  audit Domain',
 'Data Readiness:  payments Domain',
 'Data Readiness:  AQP-outputs Domain',
 'Data Readiness:  radar Domain',
 'Data Readiness:  Tableau_Usage_Data Domain',
 'Data Readiness:  SalesForce Domain',
 'Data Readiness:  Community_Zoom_Data Domain',
 'Platform: Bugs',
 'Keeping the Lights On: Pseudo Epic for Milestones and Dependencies',
 'Data Readiness: Audit Connect Domain ',
 'Data Readiness: SAQ Connect Domain ',
 'Data Readiness: Logic

In [36]:
# Get the lookup of what epic a card maps to, even if it is a nested subtask
card_epic_df=uj.get_epic_of_nested_cards(mytickets_succinct)
card_epic_df.head()

1079


,key,epic_key,epic_title
0,DATA-2,DATA-10,Data Readiness: General
1,DATA-4,DATA-10,Data Readiness: General
2,DATA-5,DATA-12,Platform: General Tasks
3,DATA-7,DATA-10,Data Readiness: General
4,DATA-8,DATA-1064,Data Readiness: user-management Domain


## Create a data frame holding cards from Data Platform Board

In [37]:
# Turn JIRA board into a df and match in the EPIC info
df=pd.DataFrame(mytickets_succinct).merge(card_epic_df,how='left',on='key') 

# Then derive the macro strand that a ticket belongs to
df.loc[df['epic_key'].notna(),'Strategy Macro Strand']=df.loc[df['epic_key'].notna(),'epic_title'].apply(lambda x: x.split(':')[0])
df.loc[df['epic_key'].notna(),'Strategy Micro Strand']=df.loc[df['epic_key'].notna(),'epic_title'].apply(lambda x: x.split(':')[1].strip() if len(x.split(':'))>1 else 'Unknown')


In [38]:
df.loc[df['key']=='DATA-1767']

,id,key,title,assignee_name,status_name,status_id,issuetype_name,issuetype_id,project_id,project_key,...,parent_id,parent_key,parent_title,parent_type,parent_type_id,description,epic_key,epic_title,Strategy Macro Strand,Strategy Micro Strand
1517,69544,DATA-1767,Data Readiness: Audit Connect Domain,None,To Do,12057,Epic,10590,12757,DATA,...,NaN,NaN,NaN,NaN,NaN,"Ingest, Cleansing and general Data Prep for Au...",NaN,NaN,NaN,NaN


In [39]:
# Show orphaned tickets
df.loc[(df['Strategy Macro Strand'].isna())&(df['issuetype_name']!='Epic')]

,id,key,title,assignee_name,status_name,status_id,issuetype_name,issuetype_id,project_id,project_key,...,parent_id,parent_key,parent_title,parent_type,parent_type_id,description,epic_key,epic_title,Strategy Macro Strand,Strategy Micro Strand
167,64874,DATA-368,Use Stitch to set up regular ingest of Tableau...,None,To Do,12057,Task,10588,12757,DATA,...,NaN,NaN,NaN,NaN,NaN,Implement the data ingest of the following dat...,NaN,NaN,NaN,NaN
168,64875,DATA-369,Ingest Stitch feed(s) into S3 for Tableau Prod...,None,To Do,12057,Task,10588,12757,DATA,...,NaN,NaN,NaN,NaN,NaN,The feed(s) need to be first stored in S3- agn...,NaN,NaN,NaN,NaN
169,64876,DATA-370,Tableau Prod (Staff) Usage Data: Build Stage,None,To Do,12057,Task,10588,12757,DATA,...,NaN,NaN,NaN,NaN,NaN,Consume as is from S3 and unpack JSON (if in J...,NaN,NaN,NaN,NaN
170,64877,DATA-371,Tableau Prod (Staff) Usage Data: Build Transform,None,To Do,12057,Task,10588,12757,DATA,...,NaN,NaN,NaN,NaN,NaN,Apply Datasource-agnostic transformations such...,NaN,NaN,NaN,NaN
171,64878,DATA-372,Tableau Prod (Staff) Usage Data: Build Logical...,None,To Do,12057,Task,10588,12757,DATA,...,NaN,NaN,NaN,NaN,NaN,Apply Datasource-specific transformations and ...,NaN,NaN,NaN,NaN
172,64879,DATA-373,Tableau Prod (Staff) Usage Data: Build Present...,None,To Do,12057,Task,10588,12757,DATA,...,NaN,NaN,NaN,NaN,NaN,Build the presentation layer(s). These should...,NaN,NaN,NaN,NaN
173,64880,DATA-374,Tableau Prod (Staff) Usage Data: Final Reconci...,None,To Do,12057,Task,10588,12757,DATA,...,NaN,NaN,NaN,NaN,NaN,Data should have been sanity-checked along the...,NaN,NaN,NaN,NaN
176,64883,DATA-377,Promote all layers to Production (i.e. release...,None,To Do,12057,Task,10588,12757,DATA,...,NaN,NaN,NaN,NaN,NaN,"If all previous steps have been done, this sho...",NaN,NaN,NaN,NaN
226,61488,DATA-440,Investigation: find out how to get data from t...,None,To Do,12057,Story,10587,12757,DATA,...,NaN,NaN,NaN,NaN,NaN,A CMM machine learning app was developed as pa...,NaN,NaN,NaN,NaN
227,61489,DATA-441,Investigation: find out how to get the data fr...,None,To Do,12057,Story,10587,12757,DATA,...,NaN,NaN,NaN,NaN,NaN,Instilled is the platform used for auditor tra...,NaN,NaN,NaN,NaN


In [40]:
#df.loc[df['key']=='DATA-1780']

### Filter out orphans, and those "no longer required"

In [41]:
df=df.loc[(df['Strategy Macro Strand'].notna())&(df['issuetype_name']!='Epic')&(df['status_name']!='No longer relevant')]
# Epics are screened out by this ^^^

In [42]:

def extract_dataset(title):

    # End to end
    if re.findall('(.*) End-to-end',title):
        return re.findall('(.*) End-to-end',title)[0]
    
    # S3, Stage, Transform, Logic, Presentation
    elif re.findall('(.*) \(Build .*\)',title):
        return re.findall('(.*) \(Build .*\)',title)[0]
    
    # Reconciliation
    elif re.findall('(.*) \(Final counts reconciliation\)',title):
        return re.findall('(.*) \(Final counts reconciliation\)',title)[0]
    
df['Dataset']=df['title'].apply(extract_dataset)
df['step_list']=df['labels'].apply(lambda x:[i for i in x if re.match('\d{1,2}_.*',i)])
df['Step']=df['step_list'].apply(lambda x:x[0] if len(x)>0 else None)


# Get the JIRA board for Boonta Eve

In [43]:
# Get project and board
be_proj,be_board=uj.myprojectAndBoard(jira,'Connect',170)

# Get tickets
be_tickets=uj.get_all_project_issues_uncapped(jira,be_proj['id'])

# Summarise tickets (removing unused fields)
be_tickets_succinct=uj.get_succint_ticket_list(be_tickets)

### Map Epics to their Macro strategy strands

In [44]:
be_board_epics,be_epic_name_list, be_epic_key_list,be_board_epic_key_lookup=uj.get_epics_only(be_tickets_succinct)

In [45]:
# Get the lookup of what epic a card maps to, even if it is a nested subtask
be_card_epic_df=uj.get_epic_of_nested_cards(be_tickets_succinct)
be_card_epic_df.head()

210


,key,epic_key,epic_title
0,CN-71,CN-93,Logging and Monitoring for Connect
1,CN-90,CN-93,Logging and Monitoring for Connect
2,CN-91,CN-93,Logging and Monitoring for Connect
3,CN-92,CN-93,Logging and Monitoring for Connect
4,CN-95,CN-146,Invite MVP


### Create a data frame holding cards from Boonta Eve Board

In [46]:
# Turn JIRA board into a df and match in the EPIC info
be_df=pd.DataFrame(be_tickets_succinct).merge(be_card_epic_df,how='left',on='key') 
#be_df.head()

In [47]:
# Get the lookup of what epic a card maps to, even if it is a nested subtask
be_card_epic_df=uj.get_epic_of_nested_cards(be_tickets_succinct)
be_card_epic_df.head()

210


,key,epic_key,epic_title
0,CN-71,CN-93,Logging and Monitoring for Connect
1,CN-90,CN-93,Logging and Monitoring for Connect
2,CN-91,CN-93,Logging and Monitoring for Connect
3,CN-92,CN-93,Logging and Monitoring for Connect
4,CN-95,CN-146,Invite MVP


### Create a data frame holding cards from Boonta Eve Board

In [48]:
# Turn JIRA board into a df and match in the EPIC info
be_df=pd.DataFrame(be_tickets_succinct).merge(be_card_epic_df,how='left',on='key') 
#be_df.head()

In [49]:
# A lot of stuff going on in Boonta Eve, grab the epics we care about. Manually assign their Roadmap items
be_epics_of_note={'CN-1463':'Data Utilisation|New & Premium Reporting'
,'CN-1600':'Data Utilisation|New & Premium Reporting'
,'CN-1775':'Data Utilisation|Second & Third Party Data'
,'CN-2227':'Data Utilisation|Customer API'
,'CN-2424':'Keeping the Lights On|SQL Server Migration'
,'CN-2495':'Data Utilisation|Second & Third Party Data'
,'CN-2510':'Data Utilisation|Data in Connect'
,'CN-2527':'Keeping the Lights On|Report Migration'
,'CN-2489':'Keeping the Lights On|Report Migration'
,'CN-2521':'Data Utilisation|Second & Third Party Data'
,'CN-2581':'Keeping the Lights On|Report Migration'
                 ,'CN-2917':'Keeping the Lights On|Report Migration'
                 }


# Filter to just these epics
be_df=be_df.loc[be_df['epic_key'].isin(be_epics_of_note.keys())]

# Map Strategy Macro/Micro Strand using manually mapping
be_df['Strategy Macro Strand']=be_df['epic_key'].apply(lambda x: be_epics_of_note[x].split('|')[0])
be_df['Strategy Micro Strand']=be_df['epic_key'].apply(lambda x: be_epics_of_note[x].split('|')[1])

# Get the JIRA board for Internal Analytics

In [50]:
# Get project and board
ia_proj,ia_board=uj.myprojectAndBoard(jira,'Internal Analytics',215)

# Get tickets
ia_tickets=uj.get_all_project_issues_uncapped(jira,ia_proj['id'])

# Summarise tickets (removing unused fields)
ia_tickets_succinct=uj.get_succint_ticket_list(ia_tickets)

## Map Epics to their Macro strategy strands

In [51]:
ia_board_epics,ia_epic_name_list, ia_epic_key_list,ia_board_epic_key_lookup=uj.get_epics_only(ia_tickets_succinct)

In [52]:
ia_epic_name_list

['Thoughtspot Launch',
 'Thoughtspot Ongoing Improvements',
 'Self-serve tool procurement',
 'Ad-hoc support']

In [53]:
# Get the lookup of what epic a card maps to, even if it is a nested subtask
ia_card_epic_df=uj.get_epic_of_nested_cards(ia_tickets_succinct)

24


# Create a data frame holding cards from Internal Analytics Board

In [54]:
# Turn JIRA board into a df and match in the EPIC info
ia_df=pd.DataFrame(ia_tickets_succinct).merge(ia_card_epic_df,how='left',on='key') 


In [55]:
ia_df.loc[ia_df['epic_key'].isna(),['key','title']]

,key,title
37,IA-40,Thoughtspot Launch
64,IA-68,Thoughtspot Ongoing Improvements
66,IA-70,Self-serve tool procurement
79,IA-83,Ad-hoc support
92,IA-96,Snowflake reporting in ThoughtSpot
93,IA-97,Snowflake SpotApp


In [56]:
# Fill in any tickets without an EPIC as "Ad hoc"
# for i in ia_df.loc[(ia_df['epic_key'].isna())&~(ia_df['issuetype_name'].isin(['Epic','Subtask'])),['key','title']].to_dict(orient='records')[1:]:
#     print(i['key'])
#     jira.update_issue_field(i['key'], fields={'parent':{'key':'IA-83'}})


In [57]:

# Then derive the macro strand that a ticket belongs to
ia_df['Strategy Macro Strand']='Data Utilisation'

# Derive micro strand as epic name for tickets under an epic
ia_df.loc[ia_df['epic_key'].notna(),'Strategy Micro Strand']=ia_df.loc[ia_df['epic_key'].notna(),'epic_title'].apply(lambda x: 'Self-serve' if re.findall('(?:Thoughtspot)|(?:Self-serve)',x) else x)

# For epics themselves, micro strategy name is their epic
ia_df.loc[ia_df['epic_key'].isna(),'Strategy Micro Strand']=ia_df.loc[ia_df['issuetype_name']=='Epic','title'].apply(lambda x: 'Self-serve' if re.findall('(?:Thoughtspot)|(?:Self-serve)',x) else x)



In [58]:
ia_df[['Strategy Micro Strand','epic_title']].drop_duplicates()

,Strategy Micro Strand,epic_title
0,Ad-hoc support,Ad-hoc support
24,Self-serve,Thoughtspot Launch
26,Self-serve,Thoughtspot Ongoing Improvements
37,Self-serve,NaN
65,Self-serve,Self-serve tool procurement
79,Ad-hoc support,NaN
92,NaN,NaN


In [59]:
# Filter out orphans
ia_df=ia_df.loc[(ia_df['Strategy Micro Strand'].notna())&(ia_df['status_name']!='No longer relevant')]


# Merge IA, Boonta Eve & Data Platform

In [60]:
# Merge dfs
print(len(df),'+',len(ia_df),'+',len(be_df))
rdmp_df=pd.concat([df,ia_df,be_df])
print(len(rdmp_df))

# Merge lists
rdmp_tickets_succinct=mytickets_succinct+ia_tickets_succinct+be_tickets_succinct
rdmp_tickets=mytickets+ia_tickets+be_tickets

1540 + 99 + 388
2027


In [61]:
pickle.dump((rdmp_tickets,rdmp_tickets_succinct,rdmp_df), open(f"3-Jira_DataRoadmapCards.pickle", "wb" ) )

In [62]:
print('Run and Saved!')

Run and Saved!
